https://github.com/crahan/ipyfilechooser

In [ ]:
# !pip install ipyfilechooser

In [ ]:
import os
import geodemo
import ipywidgets as widgets
from ipyleaflet import WidgetControl
from ipyfilechooser import FileChooser

## Getting started with ipyfilechooser

In [ ]:
data_dir = os.path.abspath('./data')

fc = FileChooser(data_dir)
fc.use_dir_icons = True
fc.filter_pattern = ['*.shp', '*.geojson']

display(fc)

In [ ]:
print(fc.selected_path)

In [ ]:
print(fc.selected_filename)

In [ ]:
print(fc.selected)

In [ ]:
fc.reset()

## Creating a toolbar button

In [ ]:
widget_width = "250px"
padding = "0px 0px 0px 5px"  # upper, right, bottom, left

toolbar_button = widgets.ToggleButton(
    value=False,
    tooltip="Toolbar",
    icon="wrench",
    layout=widgets.Layout(width="28px", height="28px", padding=padding),
)

close_button = widgets.ToggleButton(
    value=False,
    tooltip="Close the tool",
    icon="times",
    button_style="primary",
    layout=widgets.Layout(height="28px", width="28px", padding=padding),
)

In [ ]:
toolbar = widgets.HBox([toolbar_button])
toolbar

## Adding toolbar event

In [ ]:
def toolbar_click(change):
    if change["new"]:
        toolbar.children = [toolbar_button, close_button]
    else:
        toolbar.children = [toolbar_button]
        
toolbar_button.observe(toolbar_click, "value")

In [ ]:
def close_click(change):
    if change["new"]:
        toolbar_button.close()
        close_button.close()
        toolbar.close()
        
close_button.observe(close_click, "value")
toolbar

## Adding toolbar grid

In [ ]:
rows = 2
cols = 2
grid = widgets.GridspecLayout(rows, cols, grid_gap="0px", layout=widgets.Layout(width="62px"))

icons: https://fontawesome.com/v4.7.0/icons/

In [ ]:
icons = ["folder-open", "map", "info", "question"]

for i in range(rows):
    for j in range(cols):
        grid[i, j] = widgets.Button(description="", button_style="primary", icon=icons[i*rows+j], 
                                    layout=widgets.Layout(width="28px", padding="0px"))
grid

In [ ]:
toolbar = widgets.VBox([toolbar_button])

In [ ]:
def toolbar_click(change):
    if change["new"]:
        toolbar.children = [widgets.HBox([close_button, toolbar_button]), grid]
    else:
        toolbar.children = [toolbar_button]
        
toolbar_button.observe(toolbar_click, "value")
toolbar

## Adding toolbar to ipyleaflet

In [ ]:
toolbar_ctrl = WidgetControl(widget=toolbar, position="topright")

In [ ]:
m = geodemo.Map()
m.add_control(toolbar_ctrl)
m

In [ ]:
output = widgets.Output()
output_ctrl = WidgetControl(widget=output, position="topright")

In [ ]:
buttons = widgets.ToggleButtons(
    value=None,
    options=["Apply", "Reset", "Close"],
    tooltips=["Apply", "Reset", "Close"],
    button_style="primary",
)
buttons.style.button_width = "80px"

In [ ]:
filechooser_widget = widgets.VBox([fc, buttons])

In [ ]:
def button_click(change):
    if change["new"] == "Apply" and fc.selected is not None:
        if fc.selected.endswith(".shp"):
            m.add_shapefile(fc.selected, layer_name="Shapefile")
        elif fc.selected.endswith(".geojson"):
            m.add_geojson(fc.selected, layer_name="GeoJSON")
    elif change["new"] == "Reset":
        fc.reset()
    elif change["new"] == "Close":
        fc.reset()
        m.remove_control(output_ctrl)
buttons.observe(button_click, "value")        

In [ ]:
def tool_click(b):    
    with output:
        output.clear_output()
        if b.icon == "folder-open":
            display(filechooser_widget)
            m.add_control(output_ctrl)

In [ ]:
for i in range(rows):
    for j in range(cols):
        tool = grid[i, j]
        tool.on_click(tool_click)

![](https://i.imgur.com/q4ub8uC.gif)